In [ ]:
import serial
import time

# Function to send AT command and receive response
def send_command(command):
    ser.write((command + '\r\n').encode())
    time.sleep(1)
    response = ser.read(ser.inWaiting()).decode()
    return response.strip()

# Function to read the last SMS
def read_last_sms():
    # Send command to select message storage
    response = send_command('AT+CPMS="SM"')
    if "OK" not in response:
        print("Error selecting message storage.")
        return None
    
    # Send command to list received messages
    response = send_command('AT+CMGL="REC UNREAD",1')
    if "+CMGL: " not in response:
        print("No unread messages found.")
        return None
    
    # Extract message index
    index = response.split(',')[0].split(' ')[-1]
    
    # Send command to read message
    response = send_command('AT+CMGR={}'.format(index))
    if "+CMGR: " not in response:
        print("Error reading message.")
        return None
    
    # Extract message content
    message_content = response.split('\r\n', 2)[-1].strip()
    return message_content

# Main code
if __name__ == "__main__":
    # Open serial connection to the A9G module
    ser = serial.Serial('COM5', 115200, timeout=1)

    # Wait for the module to initialize
    time.sleep(2)

    try:
        # Read the last SMS
        sms_content = read_last_sms()
        if sms_content:
            print("Last SMS content:", sms_content)
        else:
            print("Failed to read the last SMS.")

    except KeyboardInterrupt:
        print("Exiting program.")

    # Close serial connection
    ser.close()
